In [1]:
seed = 42  # for reproducibility

# Imports
import os
import sys
import enum
import copy
import random
import tempfile
import warnings
import multiprocessing
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display

# import visdom
from pathlib import Path
from tqdm import tqdm
from scipy import stats

import torch
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image
torch.manual_seed(seed)
import torchio as tio
sys.path.append(str(Path().absolute().parent))

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



In [2]:
DDSM_mass_path = Path('/home/linardos/Datasets/DDSM/Images/CBIS-DDSM/Mass')
train_path = DDSM_mass_path.joinpath('Train')

In [3]:
csv_file = pd.read_csv(Path('/home/linardos/Datasets/DDSM/calc_case_description_train_set.csv'))
new_csv_file = pd.read_csv(Path('/home/linardos/Datasets/DDSM/calc_case_description_train_set.csv'))

In [4]:
csv_file

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,P_02566,2,RIGHT,MLO,1,calcification,AMORPHOUS-PLEOMORPHIC,SEGMENTAL,5,MALIGNANT,5,Calc-Training_P_02566_RIGHT_MLO/1.3.6.1.4.1.95...,Calc-Training_P_02566_RIGHT_MLO_1/1.3.6.1.4.1....,Calc-Training_P_02566_RIGHT_MLO_1/1.3.6.1.4.1....
1542,P_02572,2,LEFT,CC,1,calcification,AMORPHOUS,CLUSTERED,0,MALIGNANT,2,Calc-Training_P_02572_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_02572_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_02572_LEFT_CC_1/1.3.6.1.4.1.95...
1543,P_02572,2,LEFT,MLO,1,calcification,AMORPHOUS,CLUSTERED,0,MALIGNANT,2,Calc-Training_P_02572_LEFT_MLO/1.3.6.1.4.1.959...,Calc-Training_P_02572_LEFT_MLO_1/1.3.6.1.4.1.9...,Calc-Training_P_02572_LEFT_MLO_1/1.3.6.1.4.1.9...
1544,P_02584,1,LEFT,CC,1,calcification,PLEOMORPHIC,SEGMENTAL,0,BENIGN,4,Calc-Training_P_02584_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_02584_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_02584_LEFT_CC_1/1.3.6.1.4.1.95...


In [5]:
csv_file['image file path'][0]

'Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.9590.100.1.2.408909860712120272633130274602115723157/1.3.6.1.4.1.9590.100.1.2.47414316010368386519740343172775938548/000000.dcm'

In [6]:
csv_file['image file path'][5]

'Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.162256682111885666305889708873412464189/1.3.6.1.4.1.9590.100.1.2.406725628213826290127343763811145520834/000000.dcm'

In [7]:
new_csv_file['image file path'] = csv_file['image file path'].str.replace('Calc-Training_','')
new_csv_file['image file path'] = new_csv_file['image file path'].str.split('/', expand=True)[0]
new_csv_file['image file path'] = 'Train/' + new_csv_file['image file path'].astype(str) + '/scan.nii.gz'
new_csv_file['image file path'] = new_csv_file['image file path'].str.replace('_LEFT','/LEFT')
new_csv_file['image file path'] = new_csv_file['image file path'].str.replace('_RIGHT','/RIGHT')
new_csv_file['cropped image file path'] = csv_file['cropped image file path'].str.replace('Calc-Training_','')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.split('/', expand=True)[0]
new_csv_file['cropped image file path'] = 'Train/' + new_csv_file['cropped image file path'].astype(str) + '/roi_calc.nii.gz'
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_1/r','/r')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_LEFT','/LEFT')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_RIGHT','/RIGHT')
new_csv_file['ROI mask file path'] = csv_file['ROI mask file path'].str.replace('Calc-Training_','')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.split('/', expand=True)[0]
new_csv_file['ROI mask file path'] = 'Train/' + new_csv_file['ROI mask file path'].astype(str) + '/mask_calc.nii.gz'
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_1/m','/m')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_LEFT','/LEFT')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_RIGHT','/RIGHT')

In [8]:
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'CC/roi_calc.nii',f'CC/roi_calc_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'CC/mask_calc.nii',f'CC/mask_calc_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'MLO/roi_calc.nii',f'MLO/roi_calc_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'MLO/mask_calc.nii',f'MLO/mask_calc_1.nii')

new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'CC/roi_calc.nii',f'CC/roi_calc_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'CC/mask_calc.nii',f'CC/mask_calc_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'MLO/roi_calc.nii',f'MLO/roi_calc_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'MLO/mask_calc.nii',f'MLO/mask_calc_1.nii')

for i in range(20):
    new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'_{i}/roi_calc.nii',f'/roi_calc_{i}.nii')
    new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'_{i}/mask_calc.nii',f'/mask_calc_{i}.nii')

new_csv_file.columns = new_csv_file.columns.str.replace(' ','_')

In [9]:
new_csv_file

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Train/P_00005/RIGHT_CC/scan.nii.gz,Train/P_00005/RIGHT_CC/roi_calc_1.nii.gz,Train/P_00005/RIGHT_CC/mask_calc_1.nii.gz
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Train/P_00005/RIGHT_MLO/scan.nii.gz,Train/P_00005/RIGHT_MLO/roi_calc_1.nii.gz,Train/P_00005/RIGHT_MLO/mask_calc_1.nii.gz
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Train/P_00007/LEFT_CC/scan.nii.gz,Train/P_00007/LEFT_CC/roi_calc_1.nii.gz,Train/P_00007/LEFT_CC/mask_calc_1.nii.gz
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Train/P_00007/LEFT_MLO/scan.nii.gz,Train/P_00007/LEFT_MLO/roi_calc_1.nii.gz,Train/P_00007/LEFT_MLO/mask_calc_1.nii.gz
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,Train/P_00008/LEFT_CC/scan.nii.gz,Train/P_00008/LEFT_CC/roi_calc_1.nii.gz,Train/P_00008/LEFT_CC/mask_calc_1.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,P_02566,2,RIGHT,MLO,1,calcification,AMORPHOUS-PLEOMORPHIC,SEGMENTAL,5,MALIGNANT,5,Train/P_02566/RIGHT_MLO/scan.nii.gz,Train/P_02566/RIGHT_MLO/roi_calc_1.nii.gz,Train/P_02566/RIGHT_MLO/mask_calc_1.nii.gz
1542,P_02572,2,LEFT,CC,1,calcification,AMORPHOUS,CLUSTERED,0,MALIGNANT,2,Train/P_02572/LEFT_CC/scan.nii.gz,Train/P_02572/LEFT_CC/roi_calc_1.nii.gz,Train/P_02572/LEFT_CC/mask_calc_1.nii.gz
1543,P_02572,2,LEFT,MLO,1,calcification,AMORPHOUS,CLUSTERED,0,MALIGNANT,2,Train/P_02572/LEFT_MLO/scan.nii.gz,Train/P_02572/LEFT_MLO/roi_calc_1.nii.gz,Train/P_02572/LEFT_MLO/mask_calc_1.nii.gz
1544,P_02584,1,LEFT,CC,1,calcification,PLEOMORPHIC,SEGMENTAL,0,BENIGN,4,Train/P_02584/LEFT_CC/scan.nii.gz,Train/P_02584/LEFT_CC/roi_calc_1.nii.gz,Train/P_02584/LEFT_CC/mask_calc_1.nii.gz


In [10]:
new_csv_file.to_csv('/home/linardos/Datasets/DDSM/processed_calc_case_description_train_set.csv')